In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
from scipy import sparse
import anndata as ad


In [ ]:
np.random.seed(453)

In [ ]:
co_expressalog_map_85 = pd.read_csv('/data/passala/Generated_Tables/Maize_Rice_integration/true_pair_dataframe_default_settigns.csv')
co_expressalog_map_85

In [ ]:
one_to_ones = pd.read_csv('/data/passala/Generated_Tables/Maize_Rice_integration/maize_rice_one_to_one.csv')
one_to_ones = one_to_ones[['maize','rice']]
one_to_ones = one_to_ones.rename(columns = {'maize':'Maize Gene','rice':'Rice Gene'})
one_to_ones['One to One'] = 'Yes'
one_to_ones

In [ ]:
#co_expressalog_map_85 = co_expressalog_map_85.merge(on = ['Maize Gene','Rice Gene'], right = one_to_ones, how = 'left')

In [ ]:
co_expressalog_map_85 = co_expressalog_map_85.drop_duplicates(subset = 'Rice Gene')
co_expressalog_map_85 = co_expressalog_map_85.drop_duplicates(subset = 'Maize Gene')
co_expressalog_map_85

In [ ]:
#co_expressalog_map_85['One to One'].value_counts()

In [ ]:
rice_sc_data = sc.read_10x_mtx('/data/passala/Collaborator_Data/GEO_sourced_single_cell_for_CRUSH/Rice_japonica/Cell_ranger_results/Rice_paper/outs/filtered_feature_bc_matrix',cache=True)

In [ ]:
maize_Data = sc.read_10x_mtx('/data/passala/Collaborator_Data/GEO_sourced_single_cell_for_CRUSH/Maize_nitrate_root_SC/Cell_ranger_output_folder/Full_output/cells_with_nitrate/outs/filtered_feature_bc_matrix', cache = True)

In [ ]:
sc.pp.filter_cells(maize_Data, min_genes=200)
sc.pp.filter_genes(maize_Data, min_cells=2)

In [ ]:
sc.pp.normalize_total(maize_Data, target_sum=1e4)
sc.pp.log1p(maize_Data)


In [ ]:

sc.pp.highly_variable_genes(maize_Data, min_mean=0.0125, max_mean=3, min_disp=0.3) # disp .5


In [ ]:
sc.set_figure_params(facecolor = 'white', figsize = (9,9))
sc.tl.pca(maize_Data, svd_solver='arpack', random_state= 200)
sc.pp.neighbors(maize_Data, n_neighbors=30, n_pcs=50, random_state= 500)
sc.tl.umap(maize_Data, random_state= 221)
sc.tl.leiden(maize_Data, random_state= 1000, resolution= .6)

sc.pl.umap(maize_Data, color = 'leiden', s=100)

In [ ]:
maize_Data.obs['Cell Cluster Annotation'] = 'Not Annotated'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'] == '5'] = 'Epidermis'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['0','2','3','10'])] = 'Cortex'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['1'])] = 'Stele'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['4'])] = 'Exodermis'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['6'])] = 'CC'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['7'])] = 'Meristem'

maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['9'])] = 'Pericycle'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['11'])] = 'Root Hair'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['12'])] = 'Endodermis'
maize_Data.obs['Cell Cluster Annotation'].loc[maize_Data.obs['leiden'].isin(['13'])] = 'Protophloem'




In [ ]:
rice_sc_data.var_names_make_unique()

In [ ]:
sc.pp.normalize_total(rice_sc_data, target_sum=1e4)

sc.pp.log1p(rice_sc_data)

In [ ]:
sc.pp.highly_variable_genes(rice_sc_data, min_mean=0.0125, max_mean=3, min_disp=.2)
#dispersion .5

In [ ]:
sc.set_figure_params(facecolor = 'white', figsize = (9,9)) # Make figures bigger and on a white background because your PI is 70 and doesn't know what a .png is


In [ ]:
sc.tl.pca(rice_sc_data, svd_solver='arpack', random_state= 3456)
sc.pp.neighbors(rice_sc_data, n_neighbors=30, n_pcs=50, random_state= 112) ##30,50 # 25,45
sc.tl.umap(rice_sc_data, random_state= 649)
sc.tl.leiden(rice_sc_data, random_state= 823, resolution= .5)

sc.pl.umap(rice_sc_data, color = 'leiden', s=50,)

In [ ]:
rice_sc_data.var_names_make_unique()

In [ ]:
rice_sc_data.obs['Cell Cluster Annotation'] = 'Not Annotated'
rice_sc_data.obs['Cell Cluster Annotation'].loc[rice_sc_data.obs['leiden'].isin(['1','11'])] = 'Stele'
rice_sc_data.obs['Cell Cluster Annotation'].loc[rice_sc_data.obs['leiden'].isin(['0','3',])] = 'Cortex'
rice_sc_data.obs['Cell Cluster Annotation'].loc[rice_sc_data.obs['leiden'].isin(['2','5','6'])] = 'Epidermis'
rice_sc_data.obs['Cell Cluster Annotation'].loc[rice_sc_data.obs['leiden'].isin(['9'])] = 'Root Hair'

types_to_include_in_mapping = ['Epidermis','Cortex','Stele', 'Root Hair']

In [ ]:
sec_rice = rice_sc_data[rice_sc_data.obs.loc[rice_sc_data.obs['Cell Cluster Annotation'].isin(types_to_include_in_mapping)].index,:]
sec_rice

In [ ]:
sc.pl.umap(sec_rice, color = ['Cell Cluster Annotation'], s=50, legend_loc = 'on data')

In [ ]:
sec_maize = maize_Data[maize_Data.obs.loc[maize_Data.obs['Cell Cluster Annotation'].isin(types_to_include_in_mapping)].index,:]


In [ ]:
sc.pl.umap(sec_maize, color = ['Cell Cluster Annotation'],palette= 'tab20', s=50, legend_loc = 'on data')

In [ ]:
sec_rice.var['Gene Name'] = sec_rice.var.index
sec_rice.var = sec_rice.var.rename(columns = {'Gene Name':'Gene ID'})
random_sec_rice = sec_rice.copy()

In [ ]:
sec_rice.var = sec_rice.var.merge(right = co_expressalog_map_85, right_on = 'Rice Gene', how = 'left', left_on= 'Gene ID')
sec_rice.var

In [ ]:
random_comexpressalog_map = co_expressalog_map_85.copy()
random_comexpressalog_map['Rice Gene'] = sec_rice.var.sample(7657, random_state = 204)['gene_ids'].to_list()


In [ ]:
random_comexpressalog_map

In [ ]:
random_sample_rice = sec_rice.var.sample(10500, random_state = 204)['gene_ids'].to_list()
random_sample_maize = sec_maize.var.sample(10500, random_state = 241).index.to_list()

random_comexpressalog_map= pd.DataFrame(columns = ['Rice Gene','Maize Gene'], data = zip(random_sample_rice,random_sample_maize))
random_comexpressalog_map

In [ ]:

random_sec_rice.var = random_sec_rice.var.merge(right = random_comexpressalog_map, right_on = 'Rice Gene', how = 'left', left_on= 'Gene ID')
random_sec_rice.var

In [ ]:
sec_rice.var = sec_rice.var.set_index('Gene ID')
sec_rice.var
random_sec_rice.var = random_sec_rice.var.set_index('Gene ID')


In [ ]:
random_sec_maize = sec_maize.copy()

In [ ]:
sec_maize.var = sec_maize.var.merge(right = co_expressalog_map_85, how = 'left', right_on= 'Maize Gene', left_on='gene_ids')
sec_maize.var
random_sec_maize.var = random_sec_maize.var.merge(right = random_comexpressalog_map, how = 'left', right_on= 'Maize Gene', left_on='gene_ids')


In [ ]:
random_sec_maize

In [ ]:
rice_var_names_with_ortholog = sec_rice.var.dropna().index
rice_version_of_maize_genes_with_ortholog = sec_maize.var.dropna()['Rice Gene'].to_list()

rice_var_names_with_ortholog
overlap_of_shared_genes = rice_var_names_with_ortholog.intersection(rice_version_of_maize_genes_with_ortholog)

maize_genes_to_keep = sec_maize.var.loc[sec_maize.var['Rice Gene'].isin(overlap_of_shared_genes)].index

In [ ]:
maize_genes_to_keep

In [ ]:
rice_var_names_with_ortholog_random = random_sec_rice.var.dropna().index
rice_version_of_maize_genes_with_ortholog_random = random_sec_maize.var.dropna()['Rice Gene'].to_list()

rice_var_names_with_ortholog
overlap_of_shared_genes_random = rice_var_names_with_ortholog_random.intersection(rice_version_of_maize_genes_with_ortholog_random)

maize_genes_to_keep_random = random_sec_maize.var.loc[random_sec_maize.var['Rice Gene'].isin(overlap_of_shared_genes_random)].index

In [ ]:
random_sec_rice.var

In [ ]:
overlap_of_shared_genes_random

In [ ]:
maize_genes_to_keep_random

In [ ]:
sec_maize.var = sec_maize.var.set_index('gene_ids')
random_sec_maize.var = random_sec_maize.var.set_index('gene_ids')


In [ ]:
sec_maize

In [ ]:
random_sec_maize

In [ ]:
maize_single_cell_trimmed_to_orthologs = sec_maize[:,maize_genes_to_keep]
maize_single_cell_trimmed_to_orthologs


maize_single_cell_trimmed_to_orthologs_random = random_sec_maize[:,maize_genes_to_keep_random]


In [ ]:
random_sec_rice.var

In [ ]:
rice_genes_to_keep = sec_rice.var.loc[sec_rice.var['Rice Gene'].isin(overlap_of_shared_genes)].index

rice_genes_to_keep_random = random_sec_rice.var.loc[random_sec_rice.var['Rice Gene'].isin(overlap_of_shared_genes_random)].index

In [ ]:
rice_genes_to_keep_random

In [ ]:
rice_genes_to_keep

In [ ]:
rice_single_cell_slimmed_to_orthologs = sec_rice[:,rice_genes_to_keep]
rice_single_cell_slimmed_to_orthologs


rice_single_cell_slimmed_to_orthologs_random = random_sec_rice[:,rice_genes_to_keep_random]


In [ ]:
maize_single_cell_trimmed_to_orthologs.var.index = maize_single_cell_trimmed_to_orthologs.var['Rice Gene'].to_list()
maize_single_cell_trimmed_to_orthologs.var

In [ ]:
maize_single_cell_trimmed_to_orthologs_random.var.index = maize_single_cell_trimmed_to_orthologs_random.var['Rice Gene'].to_list()
maize_single_cell_trimmed_to_orthologs_random.var

In [ ]:
order_of_vars = rice_single_cell_slimmed_to_orthologs.var.index.to_list()
current_order_maize_vars = maize_single_cell_trimmed_to_orthologs.var_names
panda_version_of_maize_x_data = pd.DataFrame(columns = current_order_maize_vars, data = maize_single_cell_trimmed_to_orthologs.X.todense())
re_ordered_version = panda_version_of_maize_x_data.reindex(columns = order_of_vars)
re_ordered_version = sparse.csr_matrix(re_ordered_version.values)
maize_single_cell_trimmed_to_orthologs.X = re_ordered_version
maize_single_cell_trimmed_to_orthologs.var = maize_single_cell_trimmed_to_orthologs.var.reindex(order_of_vars)

In [ ]:
maize_single_cell_trimmed_to_orthologs_random

In [ ]:
maize_single_cell_trimmed_to_orthologs_random.X

In [ ]:
order_of_vars_random = rice_single_cell_slimmed_to_orthologs_random.var.index.to_list()
current_order_maize_vars_random = maize_single_cell_trimmed_to_orthologs_random.var_names
panda_version_of_maize_x_data_random = pd.DataFrame(columns = current_order_maize_vars_random, data = maize_single_cell_trimmed_to_orthologs_random.X.todense())
re_ordered_version_random = panda_version_of_maize_x_data_random.reindex(columns = order_of_vars_random)
re_ordered_version_random = sparse.csr_matrix(re_ordered_version_random.values)
maize_single_cell_trimmed_to_orthologs_random.X = re_ordered_version_random
maize_single_cell_trimmed_to_orthologs_random.var = maize_single_cell_trimmed_to_orthologs_random.var.reindex(order_of_vars_random)

In [ ]:
import pymn
import scgen

In [ ]:
concatendated_rice_maize_dataset = maize_single_cell_trimmed_to_orthologs.concatenate(rice_single_cell_slimmed_to_orthologs, batch_key = 'Species', batch_categories = ['Maize','Rice'])
concatendated_rice_maize_dataset.var = concatendated_rice_maize_dataset.var.merge(on = ['Maize Gene','Rice Gene'], right = one_to_ones, how = 'left')
concatendated_rice_maize_dataset.var.set_index('Rice Gene', drop= False, inplace = True)
concatendated_rice_maize_dataset.var.replace(np.nan,'No', inplace= True)


In [ ]:
concatendated_rice_maize_dataset_random = maize_single_cell_trimmed_to_orthologs_random.concatenate(rice_single_cell_slimmed_to_orthologs_random, batch_key = 'Species', batch_categories = ['Maize','Rice'])
concatendated_rice_maize_dataset_random.var = concatendated_rice_maize_dataset_random.var.merge(on = ['Maize Gene','Rice Gene'], right = one_to_ones, how = 'left')
concatendated_rice_maize_dataset_random.var.set_index('Rice Gene', drop= False, inplace = True)
concatendated_rice_maize_dataset_random.var.replace(np.nan,'No', inplace= True)

In [ ]:
concatendated_rice_maize_dataset

In [ ]:
concatendated_rice_maize_dataset_random.var

In [ ]:
concatendated_rice_maize_dataset_random.var['Rice Gene'] = concatendated_rice_maize_dataset_random.var['Rice Gene'] +'Rand'
concatendated_rice_maize_dataset_random.var['Maize Gene'] = concatendated_rice_maize_dataset_random.var['Maize Gene'] +'Rand'

concatendated_rice_maize_dataset_random


In [ ]:
concatendated_rice_maize_dataset_random.var.index = concatendated_rice_maize_dataset_random.var.index +'Rand'


In [ ]:
concatendated_rice_maize_dataset.obs

In [ ]:
concatendated_rice_maize_dataset_random.obs

In [ ]:
full_combined_random_and_not_random_dataset = ad.concat([concatendated_rice_maize_dataset,concatendated_rice_maize_dataset_random], axis = 1 )

In [ ]:
full_combined_random_and_not_random_dataset.obs = concatendated_rice_maize_dataset.obs

In [ ]:
dropper_table = concatendated_rice_maize_dataset.var[['Rice Gene','Maize Gene']]
dropper_table['Drop Status'] = 'Do Not Include' 
dropper_table


In [ ]:
random_table = concatendated_rice_maize_dataset_random.var[['Rice Gene','Maize Gene']]
random_table['Drop Status'] = 'Include'
random_table

In [ ]:
full_dropper_table = pd.concat([dropper_table,random_table])
full_dropper_table

In [ ]:
7113 /800

In [ ]:
7189 

In [ ]:
9*889

In [ ]:
8*889

In [1]:
import pymn
import scgen
import pandas as pd
import scanpy as sc
import numpy as np
from scipy import sparse
import anndata as ad


Global seed set to 0
/home/passala/miniconda3/envs/Single_cell_data/lib/python3.9/site-packages/chex/_src/pytypes.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [2]:
def cell_type_score_calculator(adata):
    species = ['Rice','Maize']
    list_of_celltype_scores = []
    for celltype in ['Epidermis', 'Cortex', 'Stele', 'Root Hair']:
        target_1 = 'Rice'+'|'+celltype 
        target_2 = 'Maize'+'|'+celltype 
        peripheral_1 = adata.uns['MetaNeighborUS'].at[target_1,target_2]
        peripheral_2 = adata.uns['MetaNeighborUS'].at[target_2,target_1]
        avg_score = (peripheral_1+peripheral_2)/2
        list_of_celltype_scores.append(avg_score)
    return list_of_celltype_scores

In [ ]:
889 /4

In [ ]:
8*222*4

In [3]:
import pickle
#pickle.dump(full_combined_random_and_not_random_dataset, open('/data/passala/Generated_Tables/Maize_Rice_integration/combined_random_not_random_anndata.p','wb'))
#pickle.dump(full_dropper_table, open('/data/passala/Generated_Tables/Maize_Rice_integration/full_dropper_half_random_half_not_Random_table_checkpoint.p','wb'))
full_dropper_table = pickle.load(open('/data/passala/Generated_Tables/Maize_Rice_integration/full_dropper_half_random_half_not_Random_table_checkpoint.p', "rb"))
full_combined_random_and_not_random_dataset = pickle.load(open('/data/passala/Generated_Tables/Maize_Rice_integration/combined_random_not_random_anndata.p', "rb"))

In [4]:
full_combined_random_and_not_random_dataset.obs

,n_genes,leiden,Cell Cluster Annotation,Species
AAACCCAAGAAACACT-1-Maize,455.0,10,Cortex,Maize
AAACCCAAGAGCAGCT-1-Maize,992.0,1,Stele,Maize
AAACCCAGTGATCATC-1-Maize,636.0,1,Stele,Maize
AAACCCATCAGACCCG-1-Maize,736.0,1,Stele,Maize
AAACGAACATCGTGCG-1-Maize,911.0,5,Epidermis,Maize
...,...,...,...,...
TTTGTCATCAAACCAC-1-Rice,NaN,6,Epidermis,Rice
TTTGTCATCATTGCCC-1-Rice,NaN,3,Cortex,Rice
TTTGTCATCCCTTGTG-1-Rice,NaN,0,Cortex,Rice
TTTGTCATCGGTCCGA-1-Rice,NaN,1,Stele,Rice


In [5]:
full_dropper_table['Drop Status'].value_counts()

Do Not Include    7189
Include           7113
Name: Drop Status, dtype: int64

In [6]:
list_of_cell_type_score_lists = []
for drop_step in np.arange(222):
    print(drop_step)
    
    kept_this_round = full_dropper_table.loc[full_dropper_table['Drop Status'].isin(['Include','Include - Non Random'])].index
    slimmed_random_dataset = full_combined_random_and_not_random_dataset[:,kept_this_round]

    slimmed_random_dataset = slimmed_random_dataset.copy()

    scgen.SCGEN.setup_anndata(slimmed_random_dataset, batch_key="Species", labels_key="Cell Cluster Annotation")
    model = scgen.SCGEN(slimmed_random_dataset)
    model.train(
        max_epochs=100,
        batch_size=32,
        early_stopping=True,
        early_stopping_patience=10,
    )
    corrected_adata = model.batch_removal()
    pymn.variableGenes(corrected_adata,study_col = 'Species')
    pymn.MetaNeighborUS(corrected_adata,study_col = 'Species',ct_col = 'Cell Cluster Annotation')
    this_level_results = cell_type_score_calculator(corrected_adata)
    list_of_cell_type_score_lists.append(this_level_results)

    
    #Swap 8 fake genes for 8 real
    drop_this_round = full_dropper_table.loc[full_dropper_table['Drop Status'] == 'Include'].sample(n = 8).index
    add_this_round = full_dropper_table.loc[full_dropper_table['Drop Status'] == 'Do Not Include'].sample(n = 8).index
    full_dropper_table['Drop Status'].loc[add_this_round] = 'Include - Non Random'
    full_dropper_table['Drop Status'].loc[drop_this_round] = 'Do Not Include - Dropped'


0


/home/passala/miniconda3/envs/Single_cell_data/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=100 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 13/100:  13%|█▎        | 13/100 [03:30<23:30, 16.22s/it, loss=254, v_num=1]
Monitored metric elbo_validation did not improve in the last 10 records. Best score: 1266.320. Signaling Trainer to stop.


/home/passala/miniconda3/envs/Single_cell_data/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup       


...

28
Epoch 11/100:  11%|█         | 11/100 [03:07<25:19, 17.07s/it, loss=261, v_num=1]
Monitored metric elbo_validation did not improve in the last 10 records. Best score: 1440.300. Signaling Trainer to stop.


/home/passala/miniconda3/envs/Single_cell_data/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup       


/home/passala/miniconda3/envs/Single_cell_data/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=100 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


29
Epoch 10/100:   9%|▉         | 9/100 [02:25<24:28, 16.14s/it, loss=259, v_num=1]

In [ ]:
full_dropper_table['Drop Status'].value_counts()

In [ ]:
import pickle
pickle.dump(list_of_cell_type_score_lists, open('/data/passala/Generated_Tables/Maize_Rice_integration/second_version_ultra_detail_loss_curve_data.p','wb'))
pickle.dump(full_dropper_table, open('/data/passala/Generated_Tables/Maize_Rice_integration/full_dropper_half_random_half_not_Random_table_checkpoint_1.p','wb'))
# dropper_table = pickle.load(open('/data/passala/Generated_Tables/Maize_Rice_integration/dropper_table_checkpoint.p', "rb"))
# list_of_cell_type_score_lists = pickle.load(open('/data/passala/Generated_Tables/Maize_Rice_integration/ultra_detail_loss_curve_data.p', "rb"))

In [ ]:
full_dropper_table.loc[full_dropper_table['Drop Status'] =='Include - Non Random']

In [ ]:
dropper_table

In [ ]:
list_of_cell_type_score_lists

In [ ]:
# for drop_step in np.arange(255):
#     print(drop_step)
    
#     concatendated_rice_maize_dataset = maize_single_cell_trimmed_to_orthologs.concatenate(rice_single_cell_slimmed_to_orthologs, batch_key = 'Species', batch_categories = ['Maize','Rice'])
#     kept_this_round = dropper_table.loc[dropper_table['Drop Status'] == 'Do Not Drop'].index
#     concatendated_rice_maize_dataset = concatendated_rice_maize_dataset[:,kept_this_round]

#     concatendated_rice_maize_dataset = concatendated_rice_maize_dataset.copy()

#     scgen.SCGEN.setup_anndata(concatendated_rice_maize_dataset, batch_key="Species", labels_key="Cell Cluster Annotation")
#     model = scgen.SCGEN(concatendated_rice_maize_dataset)
#     model.train(
#         max_epochs=100,
#         batch_size=32,
#         early_stopping=True,
#         early_stopping_patience=25,
#     )
#     corrected_adata = model.batch_removal()
#     pymn.variableGenes(corrected_adata,study_col = 'Species')
#     pymn.MetaNeighborUS(corrected_adata,study_col = 'Species',ct_col = 'Cell Cluster Annotation')
#     this_level_results = cell_type_score_calculator(corrected_adata)
#     list_of_cell_type_score_lists.append(this_level_results)

#     drop_this_round = dropper_table['Drop Status'].loc[(dropper_table['One to One'] == 'Yes')&(dropper_table['Drop Status'] == 'Do Not Drop')].sample(n = 8).index
#     dropper_table['Drop Status'].loc[drop_this_round] = 'Drop'

In [ ]:
len(list_of_cell_type_score_lists)

In [ ]:
import pickle
#pickle.dump(list_of_cell_type_score_lists, open('/data/passala/Generated_Tables/Maize_Rice_integration/ultra_detail_loss_curve_data.p','wb'))
#pickle.dump(dropper_table, open('/data/passala/Generated_Tables/Maize_Rice_integration/dropper_table_checkpoint.p','wb'))
#list_of_cell_type_score_lists = pickle.load(open('/data/passala/Generated_Tables/Maize_Rice_integration/ultra_detail_loss_curve_data.p', "rb"))

In [ ]:
len(np.arange(100))

In [ ]:
cell_type_score_table = pd.DataFrame(data = list_of_cell_type_score_lists, columns= ['Epidermis', 'Cortex', 'Stele', 'Root Hair'])
cell_type_score_table['Step'] = np.arange(510)

In [ ]:
tidy_table = cell_type_score_table.melt(id_vars = 'Step')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Root Hair'])

In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Epidermis'])

In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Cortex'])

In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Stele'])

In [ ]:
cell_type_score_table['Stele Rolling 10 Average'] = cell_type_score_table['Stele'].rolling(50).mean().shift(-25)


In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Stele Rolling 10 Average'])

In [ ]:
uno_to_uno_cells_excluded = np.arange(510) *8 
uno_to_uno_cells_excluded

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
fig.patch.set_facecolor('white')
ax = sns.lineplot(x = uno_to_uno_cells_excluded, y = cell_type_score_table['Stele Rolling 10 Average']) #style = 'Gene', dashes = True
plt.ylabel("Expression Similarity Between \n Maize and Rice Stele (AUROC)", fontsize = 20)
plt.xlabel("One to One Genes Excluded", fontsize = 20)
# sns.despine()

ax.grid(False)
ax.spines["bottom"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
#ax.set_xticks([1,2,3,4], labels=['Epidermis','Stele','Cortex','Root Hair'])
#ax.set_xticklabels(['Phloem 1','Xylem','Endodermis 2','Phloem 2','Endodermis','Atrichoblasts/Columella','Trichoblasts','Meristematic/QC','Cortex'])
#plt.xlim((-7,7))
# plt.ylim((0, 1250))
#plt.title("Current Threshold is 31", fontsize = 20)
plt.xticks(fontsize = 12, rotation = 25)
#plt.yticks(fontsize = 11)
plt.savefig('/data/passala/Plots_for_projects/Coexpressolog_paper_plots/Stele_dropping_genes.svg')

In [ ]:
cell_type_score_table['Epidermis Rolling 10 Average'] = cell_type_score_table['Epidermis'].rolling(50).mean().shift(-25)


In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Epidermis Rolling 10 Average'])

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
fig.patch.set_facecolor('white')
ax = sns.lineplot(x = uno_to_uno_cells_excluded, y = cell_type_score_table['Epidermis Rolling 10 Average']) #style = 'Gene', dashes = True
plt.ylabel("Expression Similarity Between \n Maize and Rice Epidermis (AUROC)", fontsize = 20)
plt.xlabel("One to One Genes Excluded", fontsize = 20)
# sns.despine()

ax.grid(False)
ax.spines["bottom"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
#ax.set_xticks([1,2,3,4], labels=['Epidermis','Stele','Cortex','Root Hair'])
#ax.set_xticklabels(['Phloem 1','Xylem','Endodermis 2','Phloem 2','Endodermis','Atrichoblasts/Columella','Trichoblasts','Meristematic/QC','Cortex'])
#plt.xlim((-7,7))
# plt.ylim((0, 1250))
#plt.title("Current Threshold is 31", fontsize = 20)
plt.xticks(fontsize = 12, rotation = 25)
#plt.yticks(fontsize = 11)
plt.savefig('/data/passala/Plots_for_projects/Coexpressolog_paper_plots/Epidermis_dropping_genes.svg')

In [ ]:
cell_type_score_table['Root Hair Rolling 10 Average'] = cell_type_score_table['Root Hair'].rolling(50).mean().shift(-25)


In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Root Hair Rolling 10 Average'])

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
fig.patch.set_facecolor('white')
ax = sns.lineplot(x = uno_to_uno_cells_excluded, y = cell_type_score_table['Root Hair Rolling 10 Average']) #style = 'Gene', dashes = True
plt.ylabel("Expression Similarity Between \n Maize and Rice Root Hair (AUROC)", fontsize = 20)
plt.xlabel("", fontsize = 20)
# sns.despine()

ax.grid(False)
ax.spines["bottom"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
#ax.set_xticks([1,2,3,4], labels=['Epidermis','Stele','Cortex','Root Hair'])
#ax.set_xticklabels(['Phloem 1','Xylem','Endodermis 2','Phloem 2','Endodermis','Atrichoblasts/Columella','Trichoblasts','Meristematic/QC','Cortex'])
#plt.xlim((-7,7))
# plt.ylim((0, 1250))
#plt.title("Current Threshold is 31", fontsize = 20)
plt.xticks(fontsize = 0, rotation = 25)
#plt.yticks(fontsize = 11)
plt.savefig('/data/passala/Plots_for_projects/Coexpressolog_paper_plots/Root_hair_dropping_genes.svg')

In [ ]:
cell_type_score_table['Cortex Rolling 10 Average'] = cell_type_score_table['Cortex'].rolling(50).mean().shift(-25)


In [ ]:
sns.lineplot(x = np.arange(510), y = cell_type_score_table['Cortex Rolling 10 Average'])

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
fig.patch.set_facecolor('white')
ax = sns.lineplot(x = uno_to_uno_cells_excluded, y = cell_type_score_table['Cortex Rolling 10 Average']) #style = 'Gene', dashes = True
plt.ylabel("Expression Similarity Between \n Maize and Cortex (AUROC)", fontsize = 20)
plt.xlabel("", fontsize = 20)
# sns.despine()

ax.grid(False)
ax.spines["bottom"].set_color("black")
ax.spines["left"].set_color("black")
ax.spines["top"].set_color("black")
ax.spines["right"].set_color("black")
#ax.set_xticks([1,2,3,4], labels=['Epidermis','Stele','Cortex','Root Hair'])
#ax.set_xticklabels(['Phloem 1','Xylem','Endodermis 2','Phloem 2','Endodermis','Atrichoblasts/Columella','Trichoblasts','Meristematic/QC','Cortex'])
#plt.xlim((-7,7))
# plt.ylim((0, 1250))
#plt.title("Current Threshold is 31", fontsize = 20)
plt.xticks(fontsize = 0, rotation = 25)
#plt.yticks(fontsize = 11)
plt.savefig('/data/passala/Plots_for_projects/Coexpressolog_paper_plots/cortex_dropping_genes.svg')

In [ ]:
tidy_table = tidy_table.rename(columns = {'variable':'Cell Type','value':'Average AUROC Across Species'})


In [ ]:
tidy_table

In [ ]:
fig, ax = plt.subplots(figsize = (15,12))
fig.patch.set_facecolor('white')
ax = sns.lineplot(data = tidy_table, x = 'Step',y = 'Average AUROC Across Species',hue = 'Cell Type')
plt.ylabel("Average AUROC Across Species", fontsize = 20)
plt.xlabel("Step", fontsize = 20)
sns.despine()

ax.spines["bottom"].set_color("black")
ax.spines["left"].set_color("black")
plt.title("Gradual Trend Towards Lower Quality As \n We Eliminate Genes", fontsize = 20)
plt.xticks(fontsize = 11)
plt.yticks(fontsize = 11)

In [ ]:
scgen.SCGEN.setup_anndata(concatendated_rice_maize_dataset, batch_key="Species", labels_key="Cell Cluster Annotation")


In [ ]:
model = scgen.SCGEN(concatendated_rice_maize_dataset)

In [ ]:
model.train(
    max_epochs=100,
    batch_size=32,
    early_stopping=True,
    early_stopping_patience=25,
)

In [ ]:
concatendated_rice_maize_dataset

In [ ]:
corrected_adata = model.batch_removal()
corrected_adata

In [ ]:
pymn.variableGenes(corrected_adata,study_col = 'Species')
pymn.MetaNeighborUS(corrected_adata,study_col = 'Species',ct_col = 'Cell Cluster Annotation')

In [ ]:
g = pymn.plotMetaNeighborUS(corrected_adata,figsize = (20,20),cmap = 'coolwarm',fontsize = 15,show = False,)